In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader, Dataset, random_split

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import cv2
import torchvision
from torchvision import transforms as T
from torchvision import datasets, models
from torchvision.datasets import ImageFolder

import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

import wandb

# custom functions from my repo
%cd plant-pathology
from custom_functions.functions import get_images_shape, preprocess_csv
%cd ..

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# set seed for reproducibility
LUCKY_SEED = 42
torch.manual_seed(LUCKY_SEED)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(LUCKY_SEED)
np.random.seed(LUCKY_SEED)

In [ ]:
dir_path = '....' # add your images path here
test = pd.read_csv("....") # add your .csv path here

In [ ]:
class TestDataset(Dataset):
    def __init__(self, path, csv, transform = None):
        self.path = path
        self.csv = csv
        self.transform = transform
        
    def __len__(self):
        return len(self.csv)
    
    def __getitem__(self, index):
        img_path = os.path.join(self.path, self.csv.iloc[index, 0]) + ".jpg"
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            aug = self.transform(image = img)
            img = aug["image"]
            
        return img

In [ ]:
my_transform_test = A.Compose([
    A.Resize(300, 300),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

In [ ]:
test_dataset = TestDataset(dir_path, test, transform = my_transform_test)

testloader = DataLoader(test_dataset, batch_size = 16, shuffle = False)

In [ ]:
model = models.efficientnet_b3(pretrained = True)

class classifier_head(nn.Module):
    def __init__(self):
        super(classifier_head, self).__init__()
        self.fc1 = nn.Linear(1536, 128)
        self.fc2 = nn.Linear(128, 4)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p = 0.15)
        
    def forward(self, x):
        x = self.dropout(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

model.classifier = classifier_head()
model.load_state_dict(torch.load('weights.pt')) # load weights from trained model
model.eval()

In [ ]:
# get predictions
predicted = []
with torch.no_grad():
    for data in tqdm(testloader, total = len(testloader)):
        data = data.float().to(device)
        probs = model(data)
        predicted.append(np.argmax(probs.cpu(), axis = 1))

# save predictions into new .csv file       
predicted = np.concatenate(predicted)
test['predictions'] = predicted
test.to_csv('test_predictions.csv', index = False)